In [9]:
import os
import pandas as pd
import numpy as np
from tensorflow.python import pywrap_tensorflow
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py

In [14]:
def show_word_and_doc_embeddings():
    for tensor_name in var_to_shape_map:
        print("tensor_name: ", tensor_name) # get the variable name
        print(reader.get_tensor(tensor_name)) # get the variable
    
cwd=os.getcwd()
models_folder_name = os.path.join(cwd,'models')
path_to_preprocessed_texts=os.path.join(cwd,'texts','preprocessed_texts_for_doc2vec.pkl')
path_to_saved_model = os.path.join(models_folder_name,'doc2vec_recipes_checkpoint_v2_1.ckpt')
path_to_save_docs_embeddings_pkl=os.path.join(cwd,'texts','docs_extracted_features.pkl')

reader = pywrap_tensorflow.NewCheckpointReader(path_to_saved_model)
var_to_shape_map = reader.get_variable_to_shape_map() # 'var_to_shape_map' is a dictionary contains every tensor in the model
show_word_and_doc_embeddings()

doc_embeddings = reader.get_tensor('doc_embeddings')
doc_embeddings = doc_embeddings.tolist() 
    
df_preprocessed_texts = pd.read_pickle(path_to_preprocessed_texts)

tensor_name:  doc_embeddings
[[-1.67475879e+00 -1.97941452e-01 -1.35480869e+00  1.88928843e+00
  -5.40918529e-01 -6.31120920e-01 -6.90901935e-01  2.71154255e-01
   9.02723074e-01  9.28132594e-01 -1.66052079e+00  2.15891433e+00]
 [-1.87328839e+00 -2.42811179e+00  2.59910297e+00  9.43085372e-01
  -6.38717532e-01  3.00140643e+00  1.28773737e+00 -4.83209603e-02
  -2.14566827e-01  1.22528851e+00  2.50039053e+00  2.21832800e+00]
 [-8.51456344e-01 -1.75056398e+00  2.26320460e-01 -1.46804941e+00
   5.90534866e-01 -1.30607831e+00  1.60416460e+00 -9.37528610e-01
  -2.84064674e+00  1.00840497e+00  1.34095573e+00  2.05136466e+00]
 [-2.19758844e+00 -2.03913188e+00 -1.19053197e+00 -2.73897856e-01
  -5.59691072e-01  1.13889849e+00  8.33397806e-01  5.51557064e-01
  -9.86541435e-02 -1.03655420e-01  1.49797654e+00  2.66584426e-01]
 [-5.99893034e-01  9.94486094e-01 -2.02032423e+00 -7.95954108e-01
  -4.12059009e-01  2.41704035e+00 -8.97049069e-01  1.96058005e-01
  -2.89163971e+00 -6.70178175e-01 -1.082145

In [3]:
 dict_doc_embeddings = {
    'text_names': df_preprocessed_texts.text_names,
    'labels' : df_preprocessed_texts.labels,
    'doc_embeddings' : doc_embeddings
}
    

df_doc_embeddings=pd.DataFrame(dict_doc_embeddings)

In [4]:
doc_embeddings=df_doc_embeddings.doc_embeddings.values.tolist()
labels=df_doc_embeddings.labels.values.tolist()
text_names=df_doc_embeddings.text_names.values.tolist()

unique_labels=sorted(set(labels))
number_of_recipe_categories=len(unique_labels)
possible_category_indices=np.linspace(0,number_of_recipe_categories-1,number_of_recipe_categories,dtype=int)
labels2integers=dict(zip(unique_labels,possible_category_indices))

print(labels2integers)

category_indices=[]
for l in labels:
    category_index=labels2integers[l]
    category_indices.append(category_index)

{'tiramisu': 3, 'sushi': 2, 'sashimi': 0, 'steak': 1}


In [5]:
#PCA

pca = PCA(n_components=2)
pca_2D_embeddings = pca.fit_transform(doc_embeddings)

dict_PCA_2D = {
    'pca-1' : pca_2D_embeddings[:,0],
    'pca-2' : pca_2D_embeddings[:,1],
    'labels' : labels,
    'category_indices': category_indices,
    'text_names': text_names
}

df_PCA_2D=pd.DataFrame(dict_PCA_2D)

In [6]:
traces=[]

for n in range(0,number_of_recipe_categories):
    df_category=df_PCA_2D[(df_PCA_2D.category_indices==n)]
    point_name=str(df_category.labels[df_category.index[0]])
    trace = go.Scatter(
    x=df_category['pca-1'],
    y=df_category['pca-2'],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category['text_names'],
    )

    traces.append(trace)

layout = go.Layout(
    title='PCA reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [7]:
#t-SNE 2D
    
tsne2D = TSNE(perplexity=35, n_components=2, verbose=2, init='pca', n_iter=2000,
                  n_iter_without_progress=500, learning_rate=5,method='exact')
tsne_2D_embeddings = tsne2D.fit_transform(doc_embeddings)
    
dict_TSNE_2D = {
    'pca-1' : tsne_2D_embeddings[:,0],
    'pca-2' : tsne_2D_embeddings[:,1],
    'labels' : labels,
    'category_indices': category_indices,
    'text_names': text_names
}

df_TSNE_2D=pd.DataFrame(dict_TSNE_2D)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 45 / 45
[t-SNE] Mean sigma: 4.734028
[t-SNE] Iteration 50: error = 32.5090097, gradient norm = 0.6579011 (50 iterations in 0.076s)
[t-SNE] Iteration 100: error = 32.6179048, gradient norm = 0.4613324 (50 iterations in 0.039s)
[t-SNE] Iteration 150: error = 32.5258499, gradient norm = 0.4900142 (50 iterations in 0.025s)
[t-SNE] Iteration 200: error = 32.6177199, gradient norm = 0.4590930 (50 iterations in 0.027s)
[t-SNE] Iteration 250: error = 32.4940780, gradient norm = 0.5061499 (50 iterations in 0.037s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 32.494078
[t-SNE] Iteration 300: error = 0.0883904, gradient norm = 0.0020058 (50 iterations in 0.031s)
[t-SNE] Iteration 350: error = 0.0881573, gradient norm = 0.0000328 (50 iterations in 0.029s)
[t-SNE] Iteration 400: error = 0.0881572, gradient norm = 0.0000007 (50 iterations in 0.027s)
[t-SNE] Iteration 450: error = 0.08

In [8]:
traces=[]

for n in range(0,number_of_recipe_categories):
    df_category=df_TSNE_2D[(df_TSNE_2D.category_indices==n)]
    point_name=str(df_category.labels[df_category.index[0]])
    trace = go.Scatter(
    x=df_category['pca-1'],
    y=df_category['pca-2'],
    mode='markers',
    marker=dict(
        colorscale='Viridis',
        opacity=0.7,
        line=dict(color='rgb(140, 140, 170)')
    ),
    name=point_name,
    text=df_category['text_names'],
    )

    traces.append(trace)

layout = go.Layout(
    title='t-SNE reduction: 2D features',
    hovermode='closest',
    margin=dict(
       l=30,
       r=30,
       b=30,
       t=30
    ),
      scene = dict(
        xaxis = dict(
            title='pca-1',
       ),
       yaxis = dict(
           title='pca-2',
       ),
    ),
 
)

fig = go.Figure(data=traces, layout=layout)
iplot(fig)

In [15]:
dict_docs_extracted_features = {
    'text_names': text_names,
    'labels' : labels,
    'features' : doc_embeddings   
}

df_docs_extracted_features=pd.DataFrame(dict_docs_extracted_features)


print("Saving extracted features pandas dataframe to: ", path_to_save_docs_embeddings_pkl)
df_docs_extracted_features.to_pickle(path_to_save_docs_embeddings_pkl)

print("Extracted features pandas dataframe:")
df_docs_extracted_features

Saving extracted features pandas dataframe to:  /notebooks/school/text_feature_extraction/texts/docs_extracted_features.pkl
Extracted features pandas dataframe:


,features,labels,text_names
0,"[-1.674758791923523, -0.19794145226478577, -1....",sashimi,1.txt
1,"[-1.8732883930206299, -2.4281117916107178, 2.5...",sashimi,10.txt
2,"[-0.851456344127655, -1.7505639791488647, 0.22...",sashimi,2.txt
3,"[-2.1975884437561035, -2.0391318798065186, -1....",sashimi,3.txt
4,"[-0.5998930335044861, 0.9944860935211182, -2.0...",sashimi,4.txt
5,"[-2.187196731567383, 1.5903836488723755, 2.276...",sashimi,5.txt
6,"[-2.919644832611084, 0.3193995952606201, -1.91...",sashimi,6.txt
7,"[-1.1364532709121704, 1.8237488269805908, 0.21...",sashimi,7.txt
8,"[-2.759977340698242, 1.0077598094940186, 0.915...",sashimi,8.txt
9,"[0.6308566331863403, 0.7117491960525513, 1.864...",sashimi,9.txt
